# Spread Modelling

In [35]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import os
import pandas as pd
import numpy as np



%matplotlib inline
mpl.rcParams['figure.figsize'] = (16, 10)
pd.set_option('display.max_rows', 500)

import plotly.graph_objects as go

# Load Data

In [36]:
df_analyse=pd.read_csv(r'C:\Users\mouni\OneDrive\Desktop\DataScience\data\processed\COVID_small_flat_table.csv',sep=';',
                       parse_dates=[0])  

df_analyse.sort_values('date',ascending=True).tail()

,date,Bhutan,Denmark,Finland
878,2022-06-18,59644,3197228,1125342
879,2022-06-19,59644,3197228,1125342
880,2022-06-20,59674,3200460,1125342
881,2022-06-21,59674,3202601,1125342
882,2022-06-22,59674,3204559,1125342


In [37]:
df_analyse['Bhutan'] = df_analyse['Bhutan'].multiply(771612)
df_analyse['Denmark'] = df_analyse['Denmark'].multiply(5831000)
df_analyse['Finland'] = df_analyse['Finland'].multiply(5531000)

In [38]:
df_analyse_1=df_analyse.dropna()

In [39]:
df_analyse_1.sort_values('date',ascending=True).tail()

,date,Bhutan,Denmark,Finland
878,2022-06-18,46022026128,18643036468000,6224266602000
879,2022-06-19,46022026128,18643036468000,6224266602000
880,2022-06-20,46045174488,18661882260000,6224266602000
881,2022-06-21,46045174488,18674366431000,6224266602000
882,2022-06-22,46045174488,18685783529000,6224266602000


# Helper Functions

In [40]:
def quick_plot(x_in, df_input,y_scale='log',slider=False):
    """ Quick basic plot for quick static evaluation of a time series
    
        you can push selective columns of your data frame by .iloc[:,[0,6,7,8]]
        
        Parameters:
        ----------
        x_in : array 
            array of date time object, or array of numbers
        df_input : pandas dataframe 
            the plotting matrix where each column is plotted
            the name of the column will be used for the legend
        scale: str
            y-axis scale as 'log' or 'linear'
        slider: bool
            True or False for x-axis slider
    
        
        Returns:
        ----------
        
    """
    fig = go.Figure()

    for each in df_input.columns:
        fig.add_trace(go.Scatter(
                        x=x_in,
                        y=df_input[each],
                        name=each,
                        opacity=0.8))
    
    fig.update_layout(autosize=True,
        width=1024,
        height=768,
        font=dict(
            family="PT Sans, monospace",
            size=18,
            color="#7f7f7f"
            )
        )
    fig.update_yaxes(type=y_scale),
    fig.update_xaxes(tickangle=-45,
                 nticks=20,
                 tickfont=dict(size=14,color="#7f7f7f")
                )
    if slider==True:
        fig.update_layout(xaxis_rangeslider_visible=True)
    fig.show()

In [41]:
quick_plot(df_analyse_1.date,
           df_analyse_1.iloc[:,1:],
           y_scale='linear',
           slider=True)

In [42]:
threshold=100

In [43]:
compare_list=[]
for pos,country in enumerate(df_analyse_1.columns[1:]):
    compare_list.append(np.array(df_analyse_1[country][df_analyse_1[country]>threshold]))

In [44]:
pd_sync_timelines=pd.DataFrame(compare_list,index=df_analyse.columns[1:]).T

In [45]:
pd_sync_timelines['date']=np.arange(pd_sync_timelines.shape[0])

In [12]:
pd_sync_timelines.head()

,Bhutan,Denmark,Finland,date
0,771612.0,5831000.0,5531000.0,0
1,771612.0,5831000.0,5531000.0,1
2,771612.0,17493000.0,5531000.0,2
3,771612.0,23324000.0,5531000.0,3
4,771612.0,23324000.0,5531000.0,4


In [13]:
quick_plot(pd_sync_timelines.date,
           pd_sync_timelines.iloc[:,:-1],
           y_scale='log',
           slider=True)

In [14]:
def doubling_rate(N_0,t,T_d):
    return N_0*np.power(2,t/T_d)

In [15]:
max_days=34

norm_slopes={
    #'doubling every day':doubling_rate(100,np.arange(10),1),
    'doubling every two days':doubling_rate(100,np.arange(20),2),
    'doubling every 4 days':doubling_rate(100,np.arange(20),4),
    'doubling every 10 days':doubling_rate(100,np.arange(20),10),
}

In [16]:
pd_sync_timelines_w_slope=pd.concat([pd.DataFrame(norm_slopes),pd_sync_timelines], axis=1)

In [17]:
pd_sync_timelines_w_slope.head(200)

,doubling every two days,doubling every 4 days,doubling every 10 days,Bhutan,Denmark,Finland,date
0,100.000000,100.000000,100.000000,771612.0,5.831000e+06,5.531000e+06,0
1,141.421356,118.920712,107.177346,771612.0,5.831000e+06,5.531000e+06,1
2,200.000000,141.421356,114.869835,771612.0,1.749300e+07,5.531000e+06,2
3,282.842712,168.179283,123.114441,771612.0,2.332400e+07,5.531000e+06,3
4,400.000000,200.000000,131.950791,771612.0,2.332400e+07,5.531000e+06,4
5,565.685425,237.841423,141.421356,771612.0,3.498600e+07,5.531000e+06,5
6,800.000000,282.842712,151.571657,771612.0,6.414100e+07,5.531000e+06,6
7,1131.370850,336.358566,162.450479,771612.0,6.414100e+07,5.531000e+06,7
8,1600.000000,400.000000,174.110113,771612.0,1.399440e+08,5.531000e+06,8
9,2262.741700,475.682846,186.606598,771612.0,1.399440e+08,5.531000e+06,9


In [18]:
quick_plot(pd_sync_timelines_w_slope.date,
           pd_sync_timelines_w_slope.iloc[:,0:6],
           y_scale='log',
           slider=True)

In [19]:
pd_sync_timelines_w_slope.to_csv(r'C:\Users\mouni\OneDrive\Desktop\DataScience\data\processed\COVID_small_sync_timeline_table.csv',sep=';',index=False)

# Scikit Linear Regression

# Prediction for Denmark

In [32]:
from sklearn.impute import SimpleImputer
from sklearn import linear_model
reg = linear_model.LinearRegression(fit_intercept=False)

In [33]:
l_vec=len(df_analyse['Denmark'])
X=np.arange(l_vec-5).reshape(-1, 1)
y=np.log(np.array(df_analyse['Denmark'][5:]))


C:\Users\mouni\AppData\Local\Temp\ipykernel_36940\2760591608.py:3: RuntimeWarning:

divide by zero encountered in log



In [ ]:
reg.fit(X,y)

In [31]:
X_hat=np.arange(l_vec).reshape(-1, 1)
Y_hat=reg_bhutan.predict(X_hat)

NameError: name 'reg_bhutan' is not defined

In [ ]:
LR_inspect=df_analyse_1[['date','Denmark']].copy()

In [ ]:
LR_inspect['prediction']=np.exp(Y_hat)

In [ ]:
quick_plot(LR_inspect.date,
           LR_inspect.iloc[:,1:],
           y_scale='log',
           slider=True)

# Piecewise Regression

In [ ]:
from sklearn import linear_model
from scipy import signal
reg = linear_model.LinearRegression(fit_intercept=True)

In [ ]:
df_analyse=pd.read_csv('../data/processed/COVID_small_flat_table.csv',sep=';',
                       parse_dates=[0])  
country_list=df_analyse.columns[1:]
df_analyse['Bhutan'] = df_analyse['Bhutan'].multiply(771612)
df_analyse['Denmark'] = df_analyse['Denmark'].multiply(5831000)
df_analyse['Finland'] = df_analyse['Finland'].multiply(5531000)

In [ ]:
df_analyse_2=df_analyse

In [ ]:
df_analyse_2.head()

In [ ]:
for each in country_list:
    df_analyse_2[each+'_filter']=signal.savgol_filter(df_analyse_2[each],
                                                     5, # window size used for filtering
                           1) # order of fitted polynomial

In [ ]:
filter_cols=['Bhutan_filter','Denmark_filter', 'Finland_filter']

In [ ]:
df_analyse_2.tail()

In [ ]:
start_pos=5
quick_plot(df_analyse_2.date[start_pos:],
           df_analyse_2[filter_cols].iloc[start_pos:,:],
           y_scale='log',
           slider=True)


In [ ]:
def get_doubling_time_via_regression(in_array):
    ''' Use a linear regression to approximate the doubling rate'''
    
    y = np.array(in_array)
    X = np.arange(-1,2).reshape(-1, 1)
    
    assert len(in_array)==3
    reg.fit(X,y)
    intercept=reg.intercept_
    slope=reg.coef_
    
    return intercept/slope

In [ ]:
def doubling_time(in_array):
    ''' Use a classical doubling time formular, 
     see https://en.wikipedia.org/wiki/Doubling_time '''
    y = np.array(in_array)
    return len(y)*np.log(2)/np.log(y[-1]/y[0])

In [ ]:
days_back = 3 # this gives a smoothing effect
for pos,country in enumerate(country_list):
    df_analyse_2[country+'_DR']=df_analyse_2[country].rolling(
                                window=days_back,
                                min_periods=days_back).apply(get_doubling_time_via_regression, raw=False)

In [ ]:
days_back = 3 # this gives a smoothing effect
for pos,country in enumerate(filter_cols):
    df_analyse_2[country+'_DR']=df_analyse_2[country].rolling(
                                window=days_back,
                                min_periods=days_back).apply(get_doubling_time_via_regression, raw=False)

In [ ]:
df_analyse_2['Denmark_DR_math']=df_analyse_2['Denmark'].rolling(
                                window=days_back,
                                min_periods=days_back).apply(doubling_time, raw=False)

In [ ]:
df_analyse_2.columns

In [ ]:
start_pos=40
quick_plot(df_analyse_2.date[start_pos:],
           df_analyse_2.iloc[start_pos:,[7,8,9,10,11,12,13]], #
           y_scale='linear',
           slider=True)

In [ ]:
start_pos=40
quick_plot(df_analyse_2.date[start_pos:],
           df_analyse_2.iloc[start_pos:,[7,8,9,10,11,12,13]],
           y_scale='linear',
           slider=True)